In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
from matplotlib.animation import FuncAnimation
from numba import jit
from time import time
from numpy.random import random
from scipy.fft import fft2, fft, fftfreq
import os

class Molecular_Dynamics:
    def __init__(self, N, L, dt, t_max, x, v, m, delta=1):
        self.L = L
        self.t_max = t_max
        self.dt = dt
        self.m = m
        self.delta = delta
        self.x = x
        self.v = v
        self.x_data = [x]
        self.v_data = [v]
        self.t = [0]
        self.pair_idx = tuple((i, i+1) for i in range(N-1))   # pair index 
        self.single_idx = (0, N-1)
        self.number_of_events = 0
        
        
    def simulation(self): 
        while self.t[-1] < t_max:
            t_event, event_idx = self.compute_next_event()
            for i in range(int(self.t[-1]/self.dt)+1, int((self.t[-1]+t_event)/dt)+1):
                self.x = np.copy(self.x + self.v * self.dt)
                self.x_data.append(self.x)
                self.v_data.append(self.v)
                self.t.append(self.t[-1] + self.dt)
                if self.t[-1] >= self.t_max: break
            self.update_v(event_idx)
            self.number_of_events += 1
            
    def get_data(self, p):
        x_data = np.array(self.x_data)[:-1]  
        v_data = np.array(self.v_data)[:-1]  
        E_data = 0.5 * self.m * v_data**2
        t = np.array(self.t)[:-1]
        if p: print("number of collisions", self.number_of_events)
        return x_data, v_data, E_data, t
        
    def pair_time(self, xi, xj, vi, vj):
        dv = vi - vj
        if dv < 0: return np.inf
        return (xj - xi - self.delta)/dv
    
    def pair_collision(self, vi, vj, mi, mj):
        vi_f = vi*(mi-mj)/(mi+mj) + vj*2*mj/(mi+mj)
        vj_f = vi*2*mi/(mi+mj) - vj*(mi-mj)/(mi+mj)
        return vi_f, vj_f
    
    def wall_time(self, x0, xN, v0, vN):
        t0 = (x0 - self.delta/2)/abs(v0) if v0 < 0 else np.inf
        tn = (L - xN - self.delta/2)/vN if vN > 0 else np.inf
        return [t0, tn]
    
    def wall_collision(self, vi):
        return -vi
    
    def compute_next_event(self):
        x, v = self.x, self.v
        pair_times = [self.pair_time(x[i], x[j], v[i], v[j]) for i, j in self.pair_idx]
        wall_times = self.wall_time(x[0], x[-1], v[0], v[-1])
        times = np.array(wall_times + pair_times)
        arg = np.argmin(times)
        t_event = times[arg]
        event_idx = self.single_idx[arg] if arg < 2 else self.pair_idx[arg-2]
        return t_event, event_idx
    
    def update_v(self, event_idx):
        v = self.v.copy()
        if type(event_idx) == tuple:
            i, j = event_idx # i and the i+1 element pair(i,j)
            v[i], v[j] = self.pair_collision(v[i], v[j], m[i], m[j])
        else:
            i = event_idx
            v[i] = self.wall_collision(v[i])
        self.v = v
        
class Monte_Carlo:
    def __init__(self, L, delta = 1, T=293.15):
        self.L = L # length of simulation box
        self.delta = delta # size of the particles
        self.T = T # temperature 
        
    def sample_configuration(self):
        x = self.sample_positions()
        N = len(x)
        m = np.ones(N)
        v = self.sample_velocities(N, m)
        print(x)
        print(N)
        return x, v, m
   
    def sample_velocities(self, N, m):
        k_b = 1.38064
        sigma = np.sqrt((k_b * self.T)/m)
        v = np.random.normal(loc = 0, scale = sigma, size=N)
        return v
    
    def sample_positions(self):
        x_positions = []
        next_position = 0.5 + np.random.poisson(1)
        while next_position < self.L - 0.5:
            if next_position < self.L/2:
                x_positions.append(next_position)
                next_position = x_positions[-1] + 1 + np.random.poisson(2)  
            else:
                x_positions.append(next_position)
                next_position = x_positions[-1] + 1 + np.random.poisson(4)
        return x_positions
    
def observables(nbins, x_data, v_data, E_data, m, N):
    x_grid = np.linspace(0, L, nbins)
    dx = x_grid[1]
    
    enumerate(x_grid)
    
    q0 = np.zeros((len(t),len(x_grid)))
    q1 = np.zeros((len(t),len(x_grid)))
    q2 = np.zeros((len(t),len(x_grid)))
    
    j0 = np.zeros((len(t),len(x_grid)))
    j1 = np.zeros((len(t),len(x_grid)))
    j2 = np.zeros((len(t),len(x_grid)))
    
    for ((x_, v_), (E_, m_)) in zip(zip(x_data.T, v_data.T), zip(E_data.T, m)):
        for i, a in enumerate(x_grid):
            idx = np.logical_and(a < x_, x_ < a + dx)
            q0[:,i] += idx.astype(int)
            q1[:,i] += v_*m_*idx.astype(int)
            q2[:,i] += E_*idx.astype(int)
            j0[:,i] += v_*idx.astype(int)
            j1[:,i] += v_*m_*v_*idx.astype(int)
            j2[:,i] += E_*v_*idx.astype(int)
    
            
    q0 = q0 / N
    q1 = q1 / N
    q2 = q2 / N
    
    j0 = j0 / N
    j1 = j1 / N
    j2 = j2 / N
    
    return x_grid, q0, q1, q2, j0, j1, j2

L = 1000
t_max = 10
dt = 0.001

MC = Monte_Carlo(L, T = 293.15)

n_MC = 2000 #Number of monte carlo simulations must change this to 1000

for i in range(n_MC):
    x, v, m = MC.sample_configuration()

    N = len(x)

    MD = Molecular_Dynamics(N, L, dt, t_max, x, v, m)
    MD.simulation()

    x_data, v_data, E_data, t = MD.get_data(p = True)

    nbins = 100

    x_grid, q0, q1, q2, j0, j1, j2 = observables(nbins, x_data, v_data, E_data, m, N)

    mid = q0.shape[1]//2 #Half of the box (position we want to correlate)#
    
    maxtime = 10000
 
    if i == 0:
        term11 = q0[0:maxtime, :] * q0[0,mid]
        term21 = q0[0:maxtime,:]
        term31 = q0[0,mid]
        
        term12 = q1[0:maxtime, :] * q1[0,mid]
        term22 = q1[0:maxtime,:]
        term32 = q1[0,mid]
        
        term13 = q2[0:maxtime, :] * q2[0,mid]
        term23 = q2[0:maxtime,:]
        term33 = q2[0,mid]
        
        term14 = j0[0:maxtime, :] * j0[0,mid]
        term24 = j0[0:maxtime,:]
        term34 = j0[0,mid]
        
        term15 = j1[0:maxtime, :] * j1[0,mid]
        term25 = j1[0:maxtime,:]
        term35 = j1[0,mid]
        
        term16 = j2[0:maxtime, :] * j2[0,mid]
        term26 = j2[0:maxtime,:]
        term36 = j2[0,mid]
        print(i)

    else:
        term11 += q0[0:maxtime, :] * q0[0,mid]
        term21 += q0[0:maxtime, :]
        term31 += q0[0,mid]
        
        term12 += q1[0:maxtime, :] * q1[0,mid]
        term22 += q1[0:maxtime, :]
        term32 += q1[0,mid]
        
        term13 += q2[0:maxtime, :] * q2[0,mid]
        term23 += q2[0:maxtime, :]
        term33 += q2[0,mid]
        
        term14 += j0[0:maxtime, :] * j0[0,mid]
        term24 += j0[0:maxtime, :]
        term34 += j0[0,mid]
        
        term15 += j1[0:maxtime, :] * j1[0,mid]
        term25 += j1[0:maxtime, :]
        term35 += j1[0,mid]
        
        term16 += j2[0:maxtime, :] * j2[0,mid]
        term26 += j2[0:maxtime, :]
        term36 += j2[0,mid]
        print(i)

C0 = (term11 / n_MC) - ((term31 / n_MC)*(term21 / n_MC))
C1 = (term12 / n_MC) - ((term32 / n_MC)*(term22 / n_MC))
C2 = (term13 / n_MC) - ((term33 / n_MC)*(term23 / n_MC))
C3 = (term14 / n_MC) - ((term34 / n_MC)*(term24 / n_MC))
C4 = (term15 / n_MC) - ((term35 / n_MC)*(term25 / n_MC))
C5 = (term16 / n_MC) - ((term36 / n_MC)*(term26 / n_MC))

T, X = np.meshgrid(t[:maxtime], x_grid)

pcm1 = plt.pcolormesh(X, T, C0.T)
plt.title('Density-Density')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

pcm1 = plt.pcolormesh(X, T, C1.T)
plt.title('Momentum-Momentum')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

pcm1 = plt.pcolormesh(X, T, C2.T)
plt.title('Energy-Energy')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

pcm1 = plt.pcolormesh(X, T, C3.T)
plt.title('Density-Density')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

pcm1 = plt.pcolormesh(X, T, C4.T)
plt.title('Momentum-Momentum')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

pcm1 = plt.pcolormesh(X, T, C5.T)
plt.title('Energy-Eenrgy')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

T, X = np.meshgrid(t[:maxtime], x_grid)

C0FT = fft2(C0)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Density-Density')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

C1FT = fft2(C1)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Momentum-Momentum')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

C2FT = fft2(C2)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Energy-Energy')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

C3FT = fft2(C3)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Density-Density')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

C4FT = fft2(C4)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Momentum-Momentum')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

C5FT = fft2(C5)
pcm2 = plt.pcolormesh(X, T,np.abs(C0FT).T, vmin = 0, vmax = 20)
plt.title('Energy-Energy')
plt.xlabel('x')
plt.ylabel('t')
plt.colorbar()
plt.show()

[0.5, 3.5, 6.5, 8.5, 10.5, 14.5, 17.5, 20.5, 22.5, 26.5, 28.5, 30.5, 34.5, 36.5, 42.5, 45.5, 48.5, 51.5, 53.5, 57.5, 60.5, 62.5, 65.5, 68.5, 72.5, 76.5, 80.5, 82.5, 85.5, 87.5, 93.5, 96.5, 98.5, 100.5, 105.5, 110.5, 114.5, 115.5, 121.5, 127.5, 128.5, 130.5, 133.5, 137.5, 140.5, 141.5, 143.5, 151.5, 154.5, 156.5, 157.5, 160.5, 164.5, 165.5, 167.5, 171.5, 174.5, 177.5, 179.5, 185.5, 192.5, 195.5, 197.5, 199.5, 203.5, 208.5, 213.5, 216.5, 220.5, 223.5, 229.5, 234.5, 237.5, 238.5, 239.5, 243.5, 245.5, 248.5, 252.5, 254.5, 256.5, 258.5, 259.5, 263.5, 269.5, 271.5, 273.5, 276.5, 278.5, 283.5, 288.5, 290.5, 293.5, 295.5, 299.5, 300.5, 301.5, 303.5, 308.5, 313.5, 320.5, 322.5, 325.5, 331.5, 335.5, 338.5, 342.5, 345.5, 348.5, 353.5, 356.5, 359.5, 361.5, 363.5, 367.5, 369.5, 372.5, 375.5, 377.5, 380.5, 384.5, 386.5, 391.5, 396.5, 398.5, 402.5, 407.5, 412.5, 415.5, 419.5, 421.5, 424.5, 428.5, 430.5, 433.5, 436.5, 440.5, 443.5, 449.5, 452.5, 455.5, 458.5, 463.5, 466.5, 468.5, 475.5, 479.5, 483.5, 

KeyboardInterrupt: 